### Check GPU configuration

In [4]:
# Check env settings
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Thu Apr  6 18:02:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Connect drive, download dataset

In [5]:
!pip install einops
!pip install visu3d
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorboardX-2.6-py2.py3-none-any.whl (114 kB)


In [6]:
from pathlib import Path
import shutil
import zipfile
import requests

# mount drive
from google.colab import drive
drive.mount('/content/drive/')
project_root_path = Path('drive/MyDrive/Colab Notebooks/3dim')

cars_train_dataset_url = 'https://drive.google.com/file/d/1bThUNtIHx4xEQyffVBSf82ABDDh2HlFn/view?usp=share_link'
ds_path = Path('datasets')
if not ds_path.exists():
  # copy dataset zip to local
  ds_path.mkdir(exist_ok=True, parents=True)
  shutil.copy(project_root_path / Path('cars_train.zip'), ds_path)

  # extract zip
  with zipfile.ZipFile(ds_path / Path('cars_train.zip'), 'r') as zip_ref:
      zip_ref.extractall(ds_path)

  DATASET_PATH = ds_path / Path('cars_train')


Mounted at /content/drive/


### Download sources from github

In [25]:
!rm -rf 3d-diffusion-pytorch
!git clone https://github.com/mryanivtal/3d-diffusion-pytorch.git

Cloning into '3d-diffusion-pytorch'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 84 (delta 43), reused 58 (delta 20), pack-reused 0
Unpacking objects: 100% (84/84), 288.22 KiB | 701.00 KiB/s, done.


### Train from scratch

In [28]:
import torch
torch.cuda.empty_cache()
!cd 3d-diffusion-pytorch; python train.py --datadir="../datasets/cars_train" --outdir="../drive/MyDrive/Colab Notebooks/3dim/output" --batchsize=32 --reportlossevery=10 --evaluateevery=1 --checkpointevery=2 --epochs=5

device: cuda
starting epoch 0
  0% 0/967 [00:00<?, ?it/s]Loss: 1.0107372999191284
  0% 0/967 [00:02<?, ?it/s]
Evaluating model...
256it [00:48,  5.32it/s]
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/einops/einops.py", line 412, in reduce
    return _apply_recipe(recipe, tensor, reduction_type=reduction)
  File "/usr/local/lib/python3.9/dist-packages/einops/einops.py", line 235, in _apply_recipe
    _reconstruct_from_shape(recipe, backend.shape(tensor))
  File "/usr/local/lib/python3.9/dist-packages/einops/einops.py", line 191, in _reconstruct_from_shape_uncached
    raise EinopsError('Shape mismatch, {} != {}'.format(length, known_product))
einops.EinopsError: Shape mismatch, 2 != 128

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/3d-diffusion-pytorch/train.py", line 151, in <module>
    img = rearrange(((img[-1].clip(-1, 1) + 1) * 127.5).astype(np.uint8), "(b a) c h w -> a 

### Train - start from checkpoint

### Inference only from model checkpoint